In [1]:
path1 = ".//Datafiniti_Hotel_Reviews_Jun19.csv"
path2 = ".//Datafiniti_Hotel_Reviews.csv"

In [2]:
import time
start = time.time()
# Import modules
import pandas as pd
from scipy import sparse
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
import re
import pickle

In [21]:
df1 = pd.read_csv(path1)
df1.drop(columns={'reviews.dateAdded'},inplace= True)
df2 = pd.read_csv(path2)
mydata = pd.concat([df1,df2])

In [24]:
# Important columns. City, Country, and Province columns are excluded because they are not reliable
imp_col_list = ['address', 'name', 'reviews.date', 'reviews.text','reviews.title']
mydatasub = mydata.loc[:,imp_col_list]

In [25]:
# Prep text, add some columns and fillna, and rename columns
mydatasub['reviews.text'] = mydatasub['reviews.text'].str.lower()
mydatasub['reviews.text'] = mydatasub['reviews.text'].replace(to_replace='[^A-Za-z0-9]+', regex=True, value=' ')
mydatasub['reviews.text'] = mydatasub['reviews.text'].fillna('')
mydatasub['review_date'] = pd.to_datetime(mydatasub['reviews.date']).dt.date
mydatasub['review_month'] = pd.to_datetime(mydatasub['reviews.date']).dt.month
mydatasub['words_in_review'] = [len(i.split()) for i in mydatasub['reviews.text']]
season_dict = {1:'Winter',2:'Winter',3:'Spring',4:'Spring',5:'Spring',6:'Summer',
               7:'Summer',8:'Summer',9:'Fall',10:'Fall',11:'Fall',12:'Winter'}
mydatasub['review_season'] = mydatasub['review_month'].map(season_dict).fillna('Summer')
mydatasub.rename(columns={'address': 'hotel_address', 'city': 'hotel_city','country':'hotel_country', 
                   'name':'hotel_name'},inplace=True)

In [27]:
mydatasub.head(3)

,hotel_address,hotel_name,reviews.date,reviews.text,reviews.title,review_date,review_month,words_in_review,review_season
0,5620 Calle Real,Best Western Plus South Coast Inn,2018-01-01T00:00:00.000Z,this hotel was nice and quiet did not know the...,Best Western Plus Hotel,2018-01-01,1,39,Winter
1,5th And San Carlos PO Box 3574,Best Western Carmel's Town House Lodge,2016-04-02T00:00:00Z,we stayed in the king suite with the separatio...,Clean rooms at solid rates in the heart of Carmel,2016-04-02,4,62,Spring
2,5th And San Carlos PO Box 3574,Best Western Carmel's Town House Lodge,2016-01-06T00:00:00Z,parking was horrible somebody ran into my rent...,Business,2016-01-06,1,55,Winter


In [28]:
mydatasub.describe()

,review_month,words_in_review
count,20000.00000,20000.000000
mean,6.42805,90.276300
std,3.18701,93.063638
min,1.00000,0.000000
25%,4.00000,45.000000
50%,6.00000,61.000000
75%,9.00000,101.000000
max,12.00000,2497.000000
